In [2]:
import os
import zarr
import timm
import random
import json
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
from pathlib import Path
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from collections import defaultdict
import sys
import torch

# import torchvision.transforms.functional as F
import random

warnings.filterwarnings("ignore")
sys.path.append("./src/")

from src.config import CFG
from src.dataloader import (
    read_zarr,
    read_info_json,
    scale_coordinates,
    create_dataset,
    create_segmentation_map,
    EziiDataset,
    drop_padding,
)
from src.network import Unet3D
from src.utils import save_images, PadToSize
from src.metric import (
    score,
    create_cls_pos,
    create_cls_pos_sikii,
    create_df,
    SegmentationLoss,
    DiceLoss,
)
from metric import visualize_epoch_results
from src.utils import save_images
from src.metric import score, create_cls_pos, create_cls_pos_sikii, create_df
from src.inference import inference, inference2pos

sample_submission = pd.read_csv("../../inputs/sample_submission.csv")

In [3]:
# padf = PadToSize(CFG.resolution)


# def last_padding(tomogram, slice_size):
#     # tomogram: (tensor)
#     b, d, h, w = tomogram.shape
#     last_padding = slice_size - d % slice_size
#     if last_padding == slice_size:
#         return tomogram
#     else:
#         return torch.cat(
#             [tomogram, torch.zeros(b, last_padding, h, w).to(tomogram.device)], dim=1
#         )


# def preprocess_tensor(tensor):
#     batch_size, depth, height, width = tensor.shape
#     tensor = tensor.unsqueeze(2)  # (b, d, h, w) -> (b, d, 1, h, w)
#     return tensor


# def inference(model, exp_name, train=True):
#     dataset = EziiDataset(
#         exp_names=[exp_name],
#         base_dir="../../inputs/train/",
#         particles_name=CFG.particles_name,
#         resolution=CFG.resolution,
#         zarr_type=["denoised"],
#         train=train,
#         slice=False,
#     )
#     res_array = CFG.original_img_shape[CFG.resolution]
#     pred_array = np.zeros(
#         (len(CFG.particles_name) + 1, res_array[0], res_array[1], res_array[2])
#     )
#     loader = DataLoader(dataset, batch_size=1, shuffle=False, pin_memory=True)
#     model.eval()
#     # tq = tqdm(loader)
#     for data in loader:  # 実験データ1つを取り出す
#         for i in range(0, data["normalized_tomogram"].shape[1], CFG.slice_):
#             normalized_tomogram = data["normalized_tomogram"][:, i : i + CFG.slice_]
#             normalized_tomogram = last_padding(normalized_tomogram, CFG.slice_)
#             normalized_tomogram = padf(normalized_tomogram)
#             normalized_tomogram = preprocess_tensor(normalized_tomogram).to("cuda")
#             pred = model(normalized_tomogram)
#             prob_pred = (
#                 torch.softmax(pred, dim=1).detach().cpu().numpy()
#             )  # torch.Size([1, 7, 32, 320, 320])
#             range_ = min(i + CFG.slice_, res_array[0])
#             hw_pad_diff = prob_pred.shape[-1] - res_array[-1]

#             if i >= res_array[0]:
#                 continue

#             if range_ == res_array[0]:
#                 pred_array[:, i:range_] += prob_pred[
#                     0, :, : res_array[0] - i, :-hw_pad_diff, :-hw_pad_diff
#                 ]
#             else:
#                 pred_array[:, i:range_] += prob_pred[
#                     0, :, :range_, :-hw_pad_diff, :-hw_pad_diff
#                 ]

#         if train:
#             segmentation_map = data["segmentation_map"]
#         else:
#             segmentation_map = None

#         normalized_tomogram = data["normalized_tomogram"]
#     # tq.close()

#     return pred_array, normalized_tomogram, segmentation_map  # (7, 92, 315, 315)


# def inference2pos(pred_segmask, exp_name):
#     import cc3d

#     cls_pos = []
#     Ascale_pos = []
#     res2ratio = CFG.resolution2ratio

#     for pred_cls in range(1, len(CFG.particles_name) + 1):
#         print(pred_cls, CFG.cls2particles[pred_cls])
#         cc, P = cc3d.connected_components(pred_segmask == pred_cls, return_N=True)
#         stats = cc3d.statistics(cc)

#         for z, y, x in stats["centroids"]:
#             Ascale_z = z * res2ratio[CFG.resolution] / res2ratio["A"]
#             Ascale_x = x * res2ratio[CFG.resolution] / res2ratio["A"]
#             Ascale_y = y * res2ratio[CFG.resolution] / res2ratio["A"]

#             cls_pos.append([pred_cls, z, y, x])
#             Ascale_pos.append([pred_cls, Ascale_z, Ascale_y, Ascale_x])

#     pred_original_df = create_df(Ascale_pos, exp_name)

#     return pred_original_df

In [4]:
train_dataset = EziiDataset(
    exp_names=CFG.train_exp_names,
    base_dir="../../inputs/train/",
    particles_name=CFG.particles_name,
    resolution=CFG.resolution,
    zarr_type=CFG.train_zarr_types,
    train=True,
    augmentation=False,
    slice=False,
    pre_read=True,
)

# train_nshuffle_dataset = EziiDataset(
#     exp_names=CFG.train_exp_names,
#     base_dir="../../inputs/train/",
#     particles_name=CFG.particles_name,
#     resolution=CFG.resolution,
#     zarr_type=CFG.train_zarr_types,
#     augmentation=False,
#     train=True,
# )

valid_dataset = EziiDataset(
    exp_names=CFG.valid_exp_names,
    base_dir="../../inputs/train/",
    particles_name=CFG.particles_name,
    resolution=CFG.resolution,
    zarr_type=CFG.valid_zarr_types,
    augmentation=False,
    train=True,
    slice=False,
    pre_read=True,
)

from tqdm import tqdm

train_loader = DataLoader(
    train_dataset,
    batch_size=CFG.batch_size,
    shuffle=True,
    drop_last=True,
    pin_memory=True,
    num_workers=CFG.num_workers,
)
# train_nshuffle_loader = DataLoader(
#     train_nshuffle_dataset,
#     batch_size=1,
#     shuffle=True,
#     drop_last=True,
#     pin_memory=True,
#     num_workers=CFG.num_workers,
# )
valid_loader = DataLoader(
    valid_dataset,
    batch_size=1,
    shuffle=False,
    pin_memory=True,
    num_workers=CFG.num_workers,
)

for data in tqdm(train_loader):
    normalized_tomogram = data["normalized_tomogram"]
    segmentation_map = data["segmentation_map"]
    break

normalized_tomogram.shape

  0%|          | 0/12 [00:19<?, ?it/s]


torch.Size([2, 184, 630, 630])

In [5]:
for data in tqdm(train_loader):
    exp_names = data["exp_name"]
    normalized_tomogram = data["normalized_tomogram"]
    segmentation_map = data["segmentation_map"]
    break

  0%|          | 0/12 [00:04<?, ?it/s]


In [6]:
exp_names, normalized_tomogram.shape, segmentation_map.shape

(['TS_86_3', 'TS_86_3'],
 torch.Size([2, 184, 630, 630]),
 torch.Size([2, 184, 630, 630]))

In [7]:
import cc3d

all_pred = []

for data in tqdm(train_dataset):
    exp_name = data["exp_name"]
    normalized_tomogram = data["normalized_tomogram"]
    segmentation_map = data["segmentation_map"]
    # print(segmentation_map.shape)
    gt = segmentation_map  # .numpy()

    cls_pos = []
    Ascale_pos = []
    res2ratio = CFG.resolution2ratio
    # exp_name = exp_names[i]

    for pred_cls in range(1, len(CFG.particles_name) + 1):
        cc, P = cc3d.connected_components(gt == pred_cls, return_N=True)
        stats = cc3d.statistics(cc)

        for z, y, x in stats["centroids"][1:]:
            Ascale_z = z * res2ratio[CFG.resolution] / res2ratio["A"]
            Ascale_x = x * res2ratio[CFG.resolution] / res2ratio["A"]
            Ascale_y = y * res2ratio[CFG.resolution] / res2ratio["A"]

            cls_pos.append([pred_cls, z, y, x])
            Ascale_pos.append([pred_cls, Ascale_z, Ascale_y, Ascale_x])

    pred_original_df = create_df(Ascale_pos, exp_name)
    all_pred.append(pred_original_df)

all_pred = pd.concat(all_pred).reset_index().drop_duplicates(subset=["x", "y", "z"])

100%|██████████| 24/24 [00:48<00:00,  2.02s/it]


In [8]:
def create_gt_df(base_dir, exp_names):
    result_df = None
    particle_names = CFG.particles_name

    for exp_name in exp_names:
        for particle in particle_names:
            np_corrds = read_info_json(
                base_dir=base_dir, exp_name=exp_name, particle_name=particle
            )  # (n, 3)
            # 各行にexp_nameとparticle_name追加
            particle_df = pd.DataFrame(np_corrds, columns=["z", "y", "x"])
            particle_df["experiment"] = exp_name
            particle_df["particle_type"] = particle

            if result_df is None:
                result_df = particle_df
            else:
                result_df = pd.concat([result_df, particle_df], axis=0).reset_index(
                    drop=True
                )

    result_df = result_df.reset_index()
    result_df = result_df[["index", "experiment", "particle_type", "x", "y", "z"]]

    return result_df


gt_df = create_gt_df("../../inputs/train/overlay/ExperimentRuns/", CFG.train_exp_names)
gt_df

,index,experiment,particle_type,x,y,z
0,0,TS_73_6,apo-ferritin,268.662,4730.318,916.115
1,1,TS_73_6,apo-ferritin,238.946,4853.061,909.898
2,2,TS_73_6,apo-ferritin,83.114,5729.560,1219.524
3,3,TS_73_6,apo-ferritin,582.143,2769.968,1076.364
4,4,TS_73_6,apo-ferritin,510.389,2157.244,362.438
...,...,...,...,...,...,...
1124,1124,TS_6_6,virus-like-particle,2609.876,4569.876,1169.759
1125,1125,TS_6_6,virus-like-particle,2213.287,4135.017,1286.851
1126,1126,TS_6_6,virus-like-particle,3303.905,5697.825,789.744
1127,1127,TS_6_6,virus-like-particle,1008.748,5949.213,1077.303


In [9]:
score(all_pred, gt_df, row_id_column_name="index", distance_multiplier=0.5, beta=4)

0.987299215367147

In [10]:
all_pred

,index,experiment,particle_type,x,y,z
0,0,TS_73_6,apo-ferritin,5395.0,4745.0,35.0
1,1,TS_73_6,apo-ferritin,5265.0,4805.0,35.0
2,2,TS_73_6,apo-ferritin,5535.0,3525.0,45.0
3,3,TS_73_6,apo-ferritin,5415.0,4625.0,45.0
4,4,TS_73_6,apo-ferritin,4635.0,2185.0,55.0
...,...,...,...,...,...,...
3963,135,TS_6_6,virus-like-particle,395.0,2705.0,965.0
3964,136,TS_6_6,virus-like-particle,1005.0,5945.0,1075.0
3965,137,TS_6_6,virus-like-particle,2605.0,4565.0,1165.0
3966,138,TS_6_6,virus-like-particle,3545.0,995.0,1195.0


In [11]:
encoder = timm.create_model(
    model_name=CFG.model_name,
    pretrained=True,
    in_chans=3,
    num_classes=0,
    global_pool="",
    features_only=True,
)
model = Unet3D(encoder=encoder).to("cuda")
model.load_state_dict(torch.load("./best_model.pth"))

<All keys matched successfully>

In [12]:
from tqdm import tqdm

# ############### validation ################
train_nshuffle_original_tomogram = defaultdict(list)
train_nshuffle_pred_tomogram = defaultdict(list)
train_nshuffle_gt_tomogram = defaultdict(list)
train_cls_pos = defaultdict(list)
train_cls_Apos = defaultdict(list)

valid_original_tomogram = defaultdict(list)
valid_pred_tomogram = defaultdict(list)
valid_gt_tomogram = defaultdict(list)
valid_cls_pos = defaultdict(list)
valid_cls_Apos = defaultdict(list)

train_mean_scores = []
valid_mean_scores = []

# for exp_name in tqdm(CFG.train_exp_names):
for exp_name in tqdm(CFG.train_exp_names[:5]):  # 5つのデータで試す
    inferenced_array, n_tomogram, segmentation_map = inference(
        model, exp_name, train=True
    )
    train_nshuffle_pred_tomogram[exp_name] = inferenced_array
    train_nshuffle_gt_tomogram[exp_name] = segmentation_map.squeeze(0)
    train_nshuffle_original_tomogram[exp_name] = n_tomogram.squeeze(0)

    mean_score, scores, pred_df, gt_df, pred_cls_pos, pred_Ascale_pos = (
        visualize_epoch_results(
            train_nshuffle_pred_tomogram,
            base_dir="../../inputs/train/overlay/ExperimentRuns/",
            sikii_dict=CFG.initial_sikii,
        )
    )
    train_cls_pos[exp_name] = pred_cls_pos
    train_cls_Apos[exp_name] = pred_Ascale_pos
    train_mean_scores.append(mean_score)
print("train_mean_scores", np.mean(train_mean_scores))

for exp_name in tqdm(CFG.valid_exp_names):
    inferenced_array, n_tomogram, segmentation_map = inference(
        model, exp_name, train=True
    )
    valid_pred_tomogram[exp_name] = inferenced_array
    valid_gt_tomogram[exp_name] = segmentation_map.squeeze(0)
    valid_original_tomogram[exp_name] = n_tomogram.squeeze(0)

    mean_score, scores, pred_df, gt_df, pred_cls_pos, pred_Ascale_pos = (
        visualize_epoch_results(
            valid_pred_tomogram,
            base_dir="../../inputs/train/overlay/ExperimentRuns/",
            sikii_dict=CFG.initial_sikii,
        )
    )
    valid_cls_pos[exp_name] = pred_cls_pos
    valid_cls_Apos[exp_name] = pred_Ascale_pos
    valid_mean_scores.append(mean_score)
print("valid_mean_scores", np.mean(valid_mean_scores))

100%|██████████| 5/5 [01:00<00:00, 12.15s/it]


train_mean_scores 0.0390025642831809


100%|██████████| 1/1 [00:11<00:00, 11.77s/it]

valid_mean_scores 0.0


In [13]:
inferenced_array, n_tomogram, segmentation_map = inference(model, exp_name, train=True)
pred_original_df = inference2pos(
    pred_segmask=inferenced_array.argmax(1), exp_name=exp_name
)
pred_df

TypeError: inference2pos() missing 1 required positional argument: 'sikii_dict'

In [ ]:
score(pred_df, gt_df, row_id_column_name="index", distance_multiplier=1.0, beta=4)

In [ ]:
pred_df[pred_df["particle_type"] == "apo-ferritin"].sort_values("z")

In [ ]:
gt_df[gt_df["particle_type"] == "apo-ferritin"].sort_values("z")

In [14]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

num_classes = len(CFG.particles_name)  # クラス数
colors = plt.cm.tab10(
    np.arange(len(CFG.particles_name))
)  # "tab10" カラーマップから色を取得

# ListedColormap を作成
class_colormap = ListedColormap(colors)


def plot_with_colormap(data, title, original_tomogram):
    masked_data = np.ma.masked_where(data <= 0, data)  # クラス0をマスク
    plt.imshow(original_tomogram, cmap="gray")
    im = plt.imshow(masked_data, cmap=class_colormap)
    plt.title(title)
    plt.axis("off")
    return im


def imshow_result(pred, gt, original, index):
    # plt.figure(figsize=(20, 5))
    ax = plt.subplot(1, 3, 1)
    plot_with_colormap(
        pred[index],
        "Train-Prediction",
        original[index],
    )
    ax = plt.subplot(1, 3, 2)
    plot_with_colormap(gt[index], "Gt", original[index])

    ax = plt.subplot(1, 3, 3)
    plt.imshow(original[index], cmap="gray")
    plt.axis("off")

    plt.show()

In [ ]:
exp_name = "TS_5_4"
index = 12
pred = train_nshuffle_pred_tomogram[exp_name].argmax(0)  # (92, 315, 315)
gt = train_nshuffle_gt_tomogram[exp_name]
original = train_nshuffle_original_tomogram[exp_name]

# imshow_result(pred, gt, original, index)

for i in range(42):
    imshow_result(pred, gt, original, i)

In [ ]:
exp_name = "TS_5_4"
index = 12
pred = train_nshuffle_pred_tomogram[exp_name].argmax(0)  # (92, 315, 315)
gt = train_nshuffle_gt_tomogram[exp_name]
original = train_nshuffle_original_tomogram[exp_name]

# imshow_result(pred, gt, original, index)

for i in range(42):
    imshow_result(pred, gt, original, i)

In [ ]:
exp_name = CFG.valid_exp_names[-1]

pred = valid_pred_tomogram[exp_name].argmax(0)
gt = valid_gt_tomogram[exp_name]
original = valid_original_tomogram[exp_name]

for i in range(42):
    imshow_result(pred, gt, original, i)

In [ ]:
exp_name = CFG.train_exp_names[-1]

pred_cls_pos = train_cls_pos[exp_name]

exp_name, np.array(pred_cls_pos).max(axis=0)

In [ ]:
pred_cls_pos

In [ ]:
exp_name = CFG.valid_exp_names[0]

pred_cls_pos = valid_cls_pos[exp_name]

np.array(pred_cls_pos).max(axis=0)

In [ ]:
gt_df

In [ ]:
score(pred_df, gt_df, row_id_column_name="index", distance_multiplier=1.0, beta=4)

In [15]:
def create_gt_df(base_dir, exp_names):
    result_df = None
    particle_names = CFG.particles_name

    for exp_name in exp_names:
        for particle in particle_names:
            np_corrds = read_info_json(
                base_dir=base_dir, exp_name=exp_name, particle_name=particle
            )  # (n, 3)
            # 各行にexp_nameとparticle_name追加
            particle_df = pd.DataFrame(np_corrds, columns=["z", "y", "x"])
            particle_df["experiment"] = exp_name
            particle_df["particle_type"] = particle

            if result_df is None:
                result_df = particle_df
            else:
                result_df = pd.concat([result_df, particle_df], axis=0).reset_index(
                    drop=True
                )

    result_df = result_df.reset_index()
    result_df = result_df[["index", "experiment", "particle_type", "x", "y", "z"]]

    return result_df

In [ ]:
# exp_name = CFG.valid_exp_names[0]
# pred = valid_pred_tomogram[exp_name].argmax(0)
# gt = valid_gt_tomogram[exp_name]
# original = valid_original_tomogram[exp_name]
import timm

encoder = timm.create_model(
    model_name=CFG.model_name,
    pretrained=True,
    in_chans=3,
    num_classes=0,
    global_pool="",
    features_only=True,
)
model = Unet3D(encoder=encoder).to("cuda")
model.load_state_dict(torch.load("./best_model.pth"))

exp_name = CFG.train_exp_names[2]
pred = train_nshuffle_pred_tomogram[exp_name].argmax(0)
gt = train_nshuffle_gt_tomogram[exp_name]
original = train_nshuffle_original_tomogram[exp_name]

base_dir = "../../inputs/train/overlay/ExperimentRuns/"
gt_df = create_gt_df(base_dir=base_dir, exp_names=[exp_name])

import cc3d

cls_pos = []
Ascale_pos = []
res2ratio = CFG.resolution2ratio

for pred_cls in range(1, len(CFG.particles_name) + 1):
    print(pred_cls, CFG.cls2particles[pred_cls])
    cc, P = cc3d.connected_components(pred == pred_cls, return_N=True)
    stats = cc3d.statistics(cc)

    for z, y, x in stats["centroids"]:
        Ascale_z = z * res2ratio[CFG.resolution] / res2ratio["A"]
        Ascale_x = x * res2ratio[CFG.resolution] / res2ratio["A"]
        Ascale_y = y * res2ratio[CFG.resolution] / res2ratio["A"]

        cls_pos.append([pred_cls, z, y, x])
        Ascale_pos.append([pred_cls, Ascale_z, Ascale_y, Ascale_x])

pred_original_df = create_df(Ascale_pos, exp_name)

score(
    pred_original_df, gt_df, row_id_column_name="index", distance_multiplier=1.0, beta=4
)

In [16]:
import timm

encoder = timm.create_model(
    model_name=CFG.model_name,
    pretrained=True,
    in_chans=3,
    num_classes=0,
    global_pool="",
    features_only=True,
)
model = Unet3D(encoder=encoder).to("cuda")
model.load_state_dict(torch.load("./best_model.pth"))

exp_name = CFG.valid_exp_names[-1]
pred = valid_pred_tomogram[exp_name]
gt = valid_gt_tomogram[exp_name]
original = valid_original_tomogram[exp_name]

base_dir = "../../inputs/train/overlay/ExperimentRuns/"
gt_df = create_gt_df(base_dir=base_dir, exp_names=[exp_name])


for constant in np.linspace(0.2, 0.9, 20):
    initial_sikii = {
        "apo-ferritin": constant,
        "beta-amylase": constant,
        "beta-galactosidase": constant,
        "ribosome": constant,
        "thyroglobulin": constant,
        "virus-like-particle": constant,
    }

    import cc3d

    cls_pos = []
    Ascale_pos = []
    res2ratio = CFG.resolution2ratio

    for pred_cls in range(1, len(CFG.particles_name) + 1):
        sikii = initial_sikii[CFG.cls2particles[pred_cls]]
        cc, P = cc3d.connected_components(pred[pred_cls] > sikii, return_N=True)
        stats = cc3d.statistics(cc)

        for z, y, x in stats["centroids"]:
            Ascale_z = z * res2ratio[CFG.resolution] / res2ratio["A"]
            Ascale_x = x * res2ratio[CFG.resolution] / res2ratio["A"]
            Ascale_y = y * res2ratio[CFG.resolution] / res2ratio["A"]

            cls_pos.append([pred_cls, z, y, x])
            Ascale_pos.append([pred_cls, Ascale_z, Ascale_y, Ascale_x])

    pred_original_df = create_df(Ascale_pos, exp_name)

    score_ = score(
        pred_original_df,
        gt_df,
        row_id_column_name="index",
        distance_multiplier=1.0,
        beta=4,
    )

    print(sikii, score_)

0.2 0.5469462401375174
0.2368421052631579 0.5871654781742873
0.2736842105263158 0.6028198405536095
0.31052631578947365 0.5922334866988573
0.34736842105263155 0.6117187294214367
0.38421052631578945 0.6305999160963964
0.42105263157894735 0.5953271994405094
0.45789473684210524 0.6337348439210094
0.49473684210526314 0.6461677902531393
0.531578947368421 0.6359704387996562
0.5684210526315789 0.6934467027554885
0.6052631578947368 0.6897470895583301
0.6421052631578947 0.6754642284092973
0.6789473684210525 0.6924073401108811
0.7157894736842105 0.7252096853643878
0.7526315789473683 0.6460158873731962
0.7894736842105263 0.6560908764505983
0.8263157894736841 0.6277111275237965
0.8631578947368421 0.5529459390187434
0.9 0.4664232085899627


In [ ]:
import timm

encoder = timm.create_model(
    model_name=CFG.model_name,
    pretrained=True,
    in_chans=3,
    num_classes=0,
    global_pool="",
    features_only=True,
)
model = Unet3D(encoder=encoder).to("cuda")
model.load_state_dict(torch.load("./best_model.pth"))

exp_name = CFG.valid_exp_names[0]
pred = valid_pred_tomogram[exp_name]
gt = valid_gt_tomogram[exp_name]
original = valid_original_tomogram[exp_name]

base_dir = "../../inputs/train/overlay/ExperimentRuns/"
gt_df = create_gt_df(base_dir=base_dir, exp_names=[exp_name])


for constant in np.linspace(0.2, 0.9, 20):
    initial_sikii = {
        "apo-ferritin": constant,
        "beta-amylase": constant,
        "beta-galactosidase": constant,
        "ribosome": constant,
        "thyroglobulin": constant,
        "virus-like-particle": constant,
    }

    import cc3d

    cls_pos = []
    Ascale_pos = []
    res2ratio = CFG.resolution2ratio

    for pred_cls in range(1, len(CFG.particles_name) + 1):
        sikii = initial_sikii[CFG.cls2particles[pred_cls]]
        cc, P = cc3d.connected_components(pred[pred_cls] > sikii, return_N=True)
        stats = cc3d.statistics(cc)

        for z, y, x in stats["centroids"]:
            Ascale_z = z * res2ratio[CFG.resolution] / res2ratio["A"]
            Ascale_x = x * res2ratio[CFG.resolution] / res2ratio["A"]
            Ascale_y = y * res2ratio[CFG.resolution] / res2ratio["A"]

            cls_pos.append([pred_cls, z, y, x])
            Ascale_pos.append([pred_cls, Ascale_z, Ascale_y, Ascale_x])

    pred_original_df = create_df(Ascale_pos, exp_name)

    score_ = score(
        pred_original_df,
        gt_df,
        row_id_column_name="index",
        distance_multiplier=1.0,
        beta=4,
    )

    print(sikii, score_)

In [ ]:
encoder = timm.create_model(
    model_name=CFG.model_name,
    pretrained=True,
    in_chans=3,
    num_classes=0,
    global_pool="",
    features_only=True,
)
model = Unet3D(encoder=encoder).to("cuda")
model.load_state_dict(torch.load("./best_model.pth"))

exp_name = CFG.valid_exp_names[0]
inferenced_array, n_tomogram, segmentation_map = inference(model, exp_name, train=True)
pred_original_df = inference2pos(
    pred_segmask=inferenced_array.argmax(0), exp_name=exp_name
)
gt_df = create_gt_df(
    base_dir="../../inputs/train/overlay/ExperimentRuns/", exp_names=[exp_name]
)

score(
    pred_original_df, gt_df, row_id_column_name="index", distance_multiplier=1.0, beta=4
)